In [1]:
pip install --upgrade fsspec==2024.10.0

  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.


In [2]:
! pip install -qU langchain_experimental langchain_openai langchain_community langchain ragas chromadb langchain-groq fastembed pypdf openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
! wget "https://arxiv.org/pdf/1810.04805.pdf"

--2024-11-01 21:01:18--  https://arxiv.org/pdf/1810.04805.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/1810.04805 [following]
--2024-11-01 21:01:18--  http://arxiv.org/pdf/1810.04805
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775166 (757K) [application/pdf]
Saving to: ‘1810.04805.pdf’

1810.04805.pdf      100%[===================>] 757.00K  --.-KB/s    in 0.04s   

2024-11-01 21:01:18 (19.1 MB/s) - ‘1810.04805.pdf’ saved [775166/775166]



# process the pdf

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("1810.04805.pdf")
documents = loader.load()

print(len(documents)) # so, the entire pdf is divided into 16 documents.

16


In [26]:
documents

[Document(metadata={'source': '1810.04805.pdf', 'page': 0}, page_content='BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout}@google.com\nAbstract\nWe introduce a new language representa-\ntion model called BERT, which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre-\nsentation models (Peters et al., 2018a; Rad-\nford et al., 2018), BERT is designed to pre-\ntrain deep bidirectional representations from\nunlabeled text by jointly conditioning on both\nleft and right context in all layers. As a re-\nsult, the pre-trained BERT model can be ﬁne-\ntuned with just one additional output layer\nto create state-of-the-art models for a wide\nrange of tasks, such as question answering and\nlanguage inference, without substantial task-\nspeciﬁc architecture modiﬁcations.\nBERT is conceptually si

In [25]:
print(type(documents))

<class 'list'>


In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                               chunk_overlap =0,
                                               length_function = len,
                                               is_separator_regex = False)
naive_chunks =  text_splitter.split_documents(documents)


In [10]:
for chunk in naive_chunks[10:15]:
  print(chunk.page_content+ "\n")


BERT BERT
E[CLS] E1  E[SEP]... EN E1’ ... EM’
C
 T1
 T[SEP]...
 TN
 T1’ ...
 TM’
[CLS] Tok 1  [SEP]... Tok N Tok 1 ... TokM
Question Paragraph
Start/End Span
BERT
E[CLS] E1  E[SEP]... EN E1’ ... EM’
C
 T1
 T[SEP]...
 TN
 T1’ ...
 TM’
[CLS] Tok 1  [SEP]... Tok N Tok 1 ... TokM
Masked Sentence A Masked Sentence B
Pre-training Fine-Tuning
NSP Mask LM Mask LM
Unlabeled Sentence A and B Pair 
SQuAD
Question Answer Pair
NERMNLI
Figure 1: Overall pre-training and ﬁne-tuning procedures for BERT. Apart from output layers, the same architec-
tures are used in both pre-training and ﬁne-tuning. The same pre-trained model parameters are used to initialize
models for different down-stream tasks. During ﬁne-tuning, all parameters are ﬁne-tuned. [CLS] is a special
symbol added in front of every input example, and [SEP] is a special separator token (e.g. separating ques-
tions/answers).
ing and auto-encoder objectives have been used
for pre-training such models (Howard and Ruder,

2018; Radford et al.,

In [18]:
pip install langchain transformers sentence_transformers

In [20]:
pip install -U langchain-huggingface

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
from google.colab import userdata
from groq import Groq
from langchain_groq import ChatGroq
#
groq_api_key = userdata.get("groq_api_key")

In [23]:
# smeantic chunking


from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

semantic_chunker = SemanticChunker(embeddings,
                                   breakpoint_threshold_type='percentile')
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in documents])

for semantic_chunk in semantic_chunks:
  if "Effect of Pre-training Tasks" in semantic_chunk.page_content:
    print(semantic_chunk.page_content)
    print(len(semantic_chunk.page_content))


Dev Set
Tasks MNLI-m QNLI MRPC SST-2 SQuAD
(Acc) (Acc) (Acc) (Acc) (F1)
BERTBASE 84.4 88.4 86.7 92.7 88.5
No NSP 83.9 84.9 86.5 92.6 87.9
LTR & No NSP 82.1 84.3 77.5 92.1 77.8
+ BiLSTM 82.1 84.1 75.7 91.6 84.9
Table 5: Ablation over the pre-training tasks using the
BERTBASE architecture. “No NSP” is trained without
the next sentence prediction task. “LTR & No NSP” is
trained as a left-to-right LM without the next sentence
prediction, like OpenAI GPT. “+ BiLSTM” adds a ran-
domly initialized BiLSTM on top of the “LTR + No
NSP” model during ﬁne-tuning. ablation studies can be found in Appendix C. 5.1 Effect of Pre-training Tasks
We demonstrate the importance of the deep bidi-
rectionality of BERT by evaluating two pre-
training objectives using exactly the same pre-
training data, ﬁne-tuning scheme, and hyperpa-
rameters as BERTBASE :
No NSP: A bidirectional model which is trained
using the “masked LM” (MLM) but without the
“next sentence prediction” (NSP) task. LTR & No NSP: A left-cont

In [27]:
from langchain_community.vectorstores import Chroma
semantic_chunk_vectorstore = Chroma.from_documents(semantic_chunks, embedding=embeddings)

In [28]:
semantic_chunk_retriever = semantic_chunk_vectorstore.as_retriever(search_kwargs={"k" : 1})
semantic_chunk_retriever.invoke("Describe the Feature-based Approach with BERT?")


[Document(metadata={}, page_content='In the table,\nMASK means that we replace the target token with\nthe [MASK] symbol for MLM; SAME means that\nwe keep the target token as is; R ND means that\nwe replace the target token with another random\ntoken. The numbers in the left part of the table repre-\nsent the probabilities of the speciﬁc strategies used\nduring MLM pre-training (BERT uses 80%, 10%,\n10%). The right part of the paper represents the\nDev set results. For the feature-based approach,\nwe concatenate the last 4 layers of BERT as the\nfeatures, which was shown to be the best approach\nin Section 5.3. From the table it can be seen that ﬁne-tuning is\nsurprisingly robust to different masking strategies.')]

In [29]:
from langchain_core.prompts import ChatPromptTemplate

rag_template =  """\
Use the following context to answer the user's query. If you cannot answer, please respond with 'I don't know'.

User's Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [34]:
chat_model = ChatGroq(temperature =0,
                      model_name = "mixtral-8x7b-32768",
                      api_key = userdata.get("groq_api_key"))


In [35]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain =(
    {
        'context':semantic_chunk_retriever,
        'question': RunnablePassthrough()
    }
    | rag_prompt| chat_model | StrOutputParser()
)


In [36]:
rag_chain.invoke("Describe the Feature-based Approach with BERT?")

'The Feature-based Approach with BERT involves using the last 4 layers of BERT as features for a downstream task. This approach was found to be the most effective in Section 5.3 of the study. During masked language modeling (MLM) pre-training, BERT uses a combination of three strategies: MASK, SAME, and R ND, with probabilities of 80%, 10%, and 10%, respectively. The MASK strategy involves replacing the target token with the [MASK] symbol, the SAME strategy keeps the target token as is, and the R ND strategy replaces the target token with another random token. The fine-tuning process for the downstream task is surprisingly robust to different masking strategies, as shown in the table.'

In [37]:
rag_chain.invoke("What is SQuADv2.0?")

'SQuAD v2.0, or Squad 2.0, is a version of the Stanford Question Answering Dataset (SQuAD) that extends the problem definition of SQuAD 1.1 by allowing for the possibility that no short answer exists in the provided paragraph. This makes the problem more realistic. To extend the SQuAD v1.1 BERT model for this task, questions that do not have an answer are treated as having an answer span with start and end at the [CLS] token, and the probability space for the start and end answer span positions is extended to include the position of the [CLS] token. For prediction, the score of the no-answer span is compared to the score of the best non-null span. The document also mentions that a BERT ensemble outperforms all existing systems by a wide margin in SQuAD 2.0, excluding entries that use BERT as one of their components.'

In [38]:
rag_chain.invoke("What is the purpose of Ablation Studies?")

'Ablation studies are used to better understand the relative importance of different components or aspects of a model. In the given context, ablation experiments are being performed on various facets of the BERT model to gain a deeper understanding of its performance.'

# Implement a RAG pipeline using Naive Chunking Strategy

In [39]:
naive_chunk_vectorstore = Chroma.from_documents(naive_chunks, embedding=embeddings)
naive_chunk_retriever = naive_chunk_vectorstore.as_retriever(search_kwargs={"k" : 5})
naive_rag_chain = (
    {"context" : naive_chunk_retriever, "question" : RunnablePassthrough()}
    | rag_prompt
    | chat_model
    | StrOutputParser()
)

In [40]:
naive_rag_chain.invoke("Describe the Feature-based Approach with BERT?")

"The Feature-based Approach with BERT involves using BERT as a feature extractor for downstream tasks. In this approach, the last 4 layers of BERT are concatenated to serve as features for a given input. This method was found to be the most effective in Section 5.3 of the study. The fine-tuning process, which adjusts the model's parameters for a specific task, has been observed to be robust against different masking strategies during the pre-training phase of BERT.\n\nIn the pre-training phase, BERT uses a masking strategy where 80% of the time, the target token is replaced with the [MASK] symbol for Masked Language Modeling (MLM), 10% of the time the target token is kept the same, and 10% of the time the target token is replaced with a random token. This strategy is used to train BERT on unlabeled data over different pre-training tasks.\n\nLater, during the fine-tuning phase, the BERT model is initialized with the pre-trained parameters, and all of the parameters are fine-tuned using 

In [41]:
naive_rag_chain.invoke("What is SQuADv2.0?")

'SQuADv2.0, or Squad 2.0, is a version of the Stanford Question Answering Dataset (SQuAD) that extends the problem definition of SQuAD 1.1 by allowing for the possibility that no short answer exists in the provided paragraph. This makes the problem more realistic. To handle questions without an answer, the SQuAD v1.1 BERT model is extended to treat such questions as having an answer span with start and end at the [CLS] token. The probability space for the start and end answer span positions is extended to include the position of the [CLS] token. During prediction, the score of the no-answer span is compared to the score of the best non-null span. The TriviaQA data used for this task consists of paragraphs from TriviaQA-Wiki formed of the first 400 tokens in documents, containing at least one of the provided possible answers.'

In [42]:
naive_rag_chain.invoke("What is the purpose of Ablation Studies?")


"Ablation studies are used to better understand the relative importance of different components or aspects of a model, such as BERT in this context. In the provided document, ablation studies are performed over several facets of BERT, including the next sentence prediction task, left-to-right language model training, and the addition of a BiLSTM during fine-tuning. The studies are designed to evaluate the impact of these components on the model's performance. Ablation studies can help researchers and developers identify which parts of a model contribute most significantly to its performance and inform future model improvements."

# Ragas Assessment Comparison for Semantic Chunker

To evaluate Our rag_chain performance we can use RAGAS.
* Ragas provide metrics like `faithfulness(how factually accurate is generated answer), answer relevancy (how relevant is the generated answer to the question) ` which can be used to understnad how well our rag_chain is generating the answers.
* metrics like `context precision(the signal to noise ratio of retrieved context), context recall(can it retrive all the relevant information required to answer the question)` helps us to understand how well our reterieving the releveant context.

To perform evaluation using RAGAS or LangSmith, we need to have a dataset with a few questions and corresponding ground truth answers.

This dataset serves as a benchmark to assess the performance of your Retrieval Augmented Generation (RAG) pipeline. RAGAS and LangSmith will use these question-answer pairs to determine how well your system retrieves relevant information and generates accurate and comprehensive answers. The ground truth answers provide the "gold standard" against which your system's output is compared.

here we don't have any predefined dataset. so we use an LLM`(grogq-mixtral-8x7b-32768) ` to synthetically generate the questions and answers and form into pairs. Later we can convert that into a huggingface dataset.

we need to use the same data on which our rag_chain is trained. LLM uses the data and form questions and answers on its own.

In [43]:
# since we are using an LLM to generate questions and answers we have to split our documents into chunks.

synthetic_data_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 0,
    length_function = len,
    is_separator_regex= False
)

synthetic_data_chunks = synthetic_data_splitter.create_documents([d.page_content for d in documents])
print(len(synthetic_data_chunks))

72


Create the dataset with following as columns

* Questions — synthetically generated (grogq-mixtral-8x7b-32768)
* Contexts — created above(Synthetic data chunks)
* Ground Truths — synthetically generated (grogq-mixtral-8x7b-32768)
* Answers — generated from our Semantic RAG Chain

In [44]:
questions=[]
ground_truths_semantic=[]
contexts = []
answers =[]

question_prompt = """\
You are a teacher preparing a test. Please create a question that can be answered by referencing the following context.

Context:
{context}
"""

question_prompt = ChatPromptTemplate.from_template(question_prompt)

ground_truth_prompt = """\
Use the following context and question to answer this question using *only* the provided context.

Question:
{question}

Context:
{context}
"""

ground_truth_prompt = ChatPromptTemplate.from_template(ground_truth_prompt)

In [46]:
question_chain = question_prompt | chat_model | StrOutputParser()

grouund_truth_chain =  ground_truth_prompt| chat_model |StrOutputParser()

In [47]:
for chunk in synthetic_data_chunks[10:20]:
  questions.append(question_chain.invoke({'context':chunk.page_content}))
  contexts.append(chunk.page_content)
  ground_truths_semantic.append(grouund_truth_chain.invoke({'question':questions[-1], "context":contexts[-1]}))
  answers.append(rag_chain.invoke(questions[-1]))

You're right, simply saying "a question" or "the section" isn't very precise. Let's break down exactly what's happening in each iteration of that `for` loop, step-by-step:

**Iteration 1:**

1. **The `chunk`:** The loop starts with the 10th chunk from your `synthetic_data_chunks`. Let's say this chunk contains the following text: "The capital of France is Paris. Paris is known for its Eiffel Tower."

2. **`questions`:** The `question_chain` uses this text ("The capital of France is Paris...") as input to the AI model. The model might generate the question: "What is the capital of France known for?"  This question is then added to the `questions` list. So now, `questions` looks like this: `["What is the capital of France known for?"]`

3. **`contexts`:**  The text of the current chunk ("The capital of France is Paris...") is added to the `contexts` list. So, `contexts` now contains: `["The capital of France is Paris..."]`

4. **`ground_truths_semantic`:** The `ground_truth_chain` uses the generated question ("What is the capital of France known for?") and the same text chunk ("The capital of France is Paris...") as input to the AI model. The model will then generate the correct answer, which might be: "Paris, the capital of France, is known for the Eiffel Tower." This answer is added to the `ground_truths_semantic` list.

5. **`answers`:** Your `rag_chain` is given the question ("What is the capital of France known for?") and it searches through your entire knowledge base to find relevant information and generate an answer. Let's say your system answers: "Paris is famous for its Eiffel Tower." This answer is added to the `answers` list.

**Iteration 2:**

The loop now moves to the 11th chunk in `synthetic_data_chunks`. Let's say this chunk contains: "Albert Einstein was a physicist. He developed the theory of relativity."

The process repeats:

1. **`questions`:**  The `question_chain` generates a question based on the new chunk, maybe: "Who developed the theory of relativity?". This is added to the `questions` list: `["What is the capital of France known for?", "Who developed the theory of relativity?"]`

2. **`contexts`:** The new chunk's text ("Albert Einstein was a physicist...") is added to the `contexts` list.

3. **`ground_truths_semantic`:** The `ground_truth_chain` generates the correct answer for the new question based on the new chunk, perhaps: "Albert Einstein developed the theory of relativity." This is added to the `ground_truths_semantic` list.

4. **`answers`:** Your `rag_chain` is given the new question ("Who developed the theory of relativity?") and generates its own answer, which is added to the `answers` list.


This continues for the remaining iterations, with each iteration processing a new chunk from your `synthetic_data_chunks`. At the end, you'll have four lists:

* `questions`:  Containing all the questions generated from the 10 chunks.
* `contexts`: Containing the text of each of those 10 chunks.
* `ground_truths_semantic`: Containing the correct answers for each question.
* `answers`:  Containing the answers your system provided for each question.


NOte: for experimental purpose we have considered onlyl 10 samples.

In [50]:
questions

['Question: What special symbols are added to the input examples in BERT and what do they represent?\n\nStart/End Span: [CLS], [SEP]\n\nExplanation: In the context provided, it is mentioned that [CLS] is a special symbol added in front of every input example, and [SEP] is a special separator token. These symbols have specific meanings and functions in the BERT model. [CLS] is used to represent the beginning of an input example, and [SEP] is used to separate different parts of the input, such as questions and answers in a question-answering task.',
 'Question: According to the passage, what is the two-step framework of BERT and what happens in each step?\n\nAnswer: The two-step framework of BERT includes pre-training and fine-tuning. During pre-training, the model is trained on unlabeled data over different pre-training tasks. For fine-tuning, the BERT model is first initialized with the pre-trained parameters, and all of the parameters are fine-tuned using labeled data from the downstr

In [52]:
# formating the content generated into a huggingface dataset format

from datasets import load_dataset, Dataset

qagc_list = []

for question, answer, context, ground_truth in zip(questions, answers, contexts, ground_truths_semantic):
  # Ensuring 'contexts' is a list
  if isinstance(context, str):  # Check if context is a string
    context = [context]       # Convert to a list if it's a string
  qagc_list.append({
      "question" : question,
      "answer" : answer,
      "contexts" : context,  # Assuming 'contexts' should be the retrieved contexts
      "ground_truth" : ground_truth,
      "retrieved_contexts": context  # Add 'retrieved_contexts' as a list
  })

eval_dataset = Dataset.from_list(qagc_list)
eval_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth', 'retrieved_contexts'],
    num_rows: 10
})

In [60]:
qagc_list

[{'question': 'Question: What special symbols are added to the input examples in BERT and what do they represent?\n\nStart/End Span: [CLS], [SEP]\n\nExplanation: In the context provided, it is mentioned that [CLS] is a special symbol added in front of every input example, and [SEP] is a special separator token. These symbols have specific meanings and functions in the BERT model. [CLS] is used to represent the beginning of an input example, and [SEP] is used to separate different parts of the input, such as questions and answers in a question-answering task.',
  'answer': 'The special symbols added to the input examples in BERT are [CLS] and [SEP]. [CLS] is added at the beginning of every input example and represents the beginning of the input. [SEP] is used to separate different parts of the input, such as questions and answers in a question-answering task.',
  'contexts': ['BERT BERT\nE[CLS] E1  E[SEP]... EN E1’ ... EM’\nC\n T1\n T[SEP]...\n TN\n T1’ ...\n TM’\n[CLS] Tok 1  [SEP]... 

In [54]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key = userdata.get("openai_api_key"))

<ipython-input-54-2e332d8551a4>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key = userdata.get("openai_api_key"))


In [55]:
# implementing ragas metrics and evaluating our created dataset.

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

#
from ragas import evaluate

result = evaluate(
    eval_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
     llm=llm,
    embeddings=embeddings,
    raise_exceptions=False
)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [56]:
result

{'context_precision': 1.0000, 'faithfulness': 0.6002, 'answer_relevancy': 0.7922, 'context_recall': 0.8083}

In [57]:
#Extract the details into a dataframe
results_df = result.to_pandas()
results_df

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,Question: What special symbols are added to th...,[BERT BERT\nE[CLS] E1 E[SEP]... EN E1’ ... EM...,The special symbols added to the input example...,The special symbols added to the input example...,1.0,1.000000,0.855733,1.000000
1,"Question: According to the passage, what is th...","[2018; Radford et al., 2018; Dai and Le, 2015)...",The user's query asks about the two-step frame...,The two-step framework of BERT includes pre-tr...,1.0,0.727273,0.745228,0.750000
2,"Question: According to the passage, what is th...","[arate ﬁne-tuned models, even though they are ...","Based on the provided context, the model archi...","According to the passage, the model architectu...",1.0,0.500000,0.852341,0.500000
3,Question:\nWhy did the developers choose BERTB...,"[H, and the number of self-attention heads as ...",The developers chose BERTBASE to have the same...,The developers chose BERTBASE to have the same...,1.0,1.000000,0.822168,0.666667
4,Question: Given the input representation guide...,[Input/Output Representations To make BERT\nha...,A pair of sentences in BERT's input representa...,A pair of sentences in BERT's input representa...,1.0,1.000000,0.759377,1.000000
5,"Question: Based on the context, what are the t...",[ding to every token indicating whether it bel...,The two unsupervised tasks used to pre-train B...,right and right-to-left model. To test this hy...,1.0,0.125000,0.711801,1.000000
6,Question:\n\nWhat is the procedure called for ...,[right and a right-to-left model. Unfortunatel...,The other common name for the procedure called...,The procedure you're referring to is commonly ...,1.0,0.000000,0.556785,0.666667
7,"Question: According to the given context, what...",[kens in each sequence at random. In contrast ...,The original i-th token is kept unchanged 10% ...,The original i-th token is kept unchanged 10% ...,1.0,0.400000,0.731638,1.000000
8,"Question: Based on the given context, what is ...",[tion Answering (QA) and Natural Language Infe...,The purpose of pre-training a model with a bin...,The purpose of pre-training a model with a bin...,1.0,1.000000,0.901197,0.500000
9,"Question: Based on the context, what are the t...",[[CLS] he likes play ##ing [SEP]my dog is cute...,"Based on the context provided, the two sources...",The two sources of data used for pre-training ...,1.0,0.250000,0.985869,1.000000


In [65]:
import os
from google.colab import userdata
import openai
os.environ['OPENAI_API_KEY'] = userdata.get('openai_api_key')
openai.api_key = os.environ['OPENAI_API_KEY']

In [66]:
from ragas import evaluate

result = evaluate(
    eval_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)
result

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 0.5850, 'answer_relevancy': 0.9095, 'context_recall': 0.8250}

# Ragas Assessment Comparison for Naive Chunker

In [58]:
import tqdm
questions = []
ground_truths_semantic = []
contexts = []
answers = []
for chunk in tqdm.tqdm(synthetic_data_chunks[10:20]):
  questions.append(question_chain.invoke({"context" : chunk.page_content}))
  contexts.append([chunk.page_content])
  ground_truths_semantic.append(grouund_truth_chain.invoke({"question" : questions[-1], "context" : contexts[-1]}))
  answers.append(naive_rag_chain.invoke(questions[-1]))

100%|██████████| 10/10 [05:49<00:00, 34.90s/it]


In [59]:
qagc_list = []

for question, answer, context, ground_truth in zip(questions, answers, contexts, ground_truths_semantic):
  qagc_list.append({
      "question" : question,
      "answer" : answer,
      "contexts" : context,
      "ground_truth" : ground_truth
  })

naive_eval_dataset = Dataset.from_list(qagc_list)
naive_eval_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 10
})

In [63]:
naive_result = evaluate(
    naive_eval_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm = llm,
    embeddings  = embeddings,
    raise_exceptions = False
)
#
naive_result

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 0.5208, 'answer_relevancy': 0.7663, 'context_recall': 0.8417}

In [67]:
naive_results_df = naive_result.to_pandas()
naive_results_df

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,Question: What special symbols are added to th...,[BERT BERT\nE[CLS] E1 E[SEP]... EN E1’ ... EM...,The special symbols added to the input example...,The special symbols added to the input example...,1.0,1.000000,0.855733,1.000000
1,"Question: According to the passage, what is th...","[2018; Radford et al., 2018; Dai and Le, 2015)...",The two-step framework of BERT includes pre-tr...,The two-step framework of BERT includes pre-tr...,1.0,0.416667,0.733429,0.750000
2,"Question: According to the passage, what is th...","[arate ﬁne-tuned models, even though they are ...",The model architecture of BERT is a multi-laye...,"According to the passage, the model architectu...",1.0,0.400000,0.616213,0.500000
3,Question:\nWhy did the developers choose BERTB...,"[H, and the number of self-attention heads as ...",The developers chose BERTBASE to have the same...,The developers chose BERTBASE to have the same...,1.0,1.000000,0.822168,0.666667
4,Question: Given the input representation guide...,[Input/Output Representations To make BERT\nha...,A pair of sentences in BERT's input representa...,A pair of sentences in BERT's input representa...,1.0,0.666667,0.612425,0.750000
5,"Question: Based on the context, what are the t...",[ding to every token indicating whether it bel...,The two unsupervised tasks used to pre-train B...,The two unsupervised tasks used to pre-train B...,1.0,0.125000,0.711801,1.000000
6,Question:\n\nWhat is the procedure called for ...,[right and a right-to-left model. Unfortunatel...,The other common name for the procedure called...,The other common name for the procedure called...,1.0,0.000000,0.596690,1.000000
7,"Question: According to the given context, what...",[kens in each sequence at random. In contrast ...,The original i-th token is kept unchanged 10% ...,The original i-th token is kept unchanged 10% ...,1.0,0.500000,0.827963,1.000000
8,"Question: Based on the given context, what is ...",[tion Answering (QA) and Natural Language Infe...,The purpose of pre-training a model with a bin...,The purpose of pre-training a model with a bin...,1.0,1.000000,0.901197,0.750000
9,"Question: Based on the context, what are the t...",[[CLS] he likes play ##ing [SEP]my dog is cute...,The two sources of data used for pre-training ...,The two sources of data used for pre-training ...,1.0,0.100000,0.985869,1.000000
